# Data Samurai - Tom Martin

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.proportion import proportions_ztest

## EDA

These first steps are just to understand the provided datasets, what values they have, and whether there are any null values. I like to check the head and tails of each dataframe to determine if there are any formatting issues.

In [2]:
calls = pd.read_csv("./data/calls.csv")
leads = pd.read_csv("./data/leads.csv")
signups = pd.read_csv("./data/signups.csv")

In [3]:
calls.head()

,Phone Number,Call Outcome,Agent,Call Number
0,83473306392,NOT INTERESTED,orange,0
1,762850680150,CALL BACK LATER,orange,1
2,476309275079,NOT INTERESTED,orange,2
3,899921761538,CALL BACK LATER,red,3
4,906739234066,CALL BACK LATER,orange,4


In [4]:
calls.tail()

,Phone Number,Call Outcome,Agent,Call Number
4995,636020617927,CALL BACK LATER,orange,4995
4996,716436659171,NOT INTERESTED,orange,4996
4997,591005517672,CALL BACK LATER,red,4997
4998,583289319218,ANSWER MACHINE,black,4998
4999,860326177102,ANSWER MACHINE,orange,4999


In [5]:
# Calls have a many-to-one relationship with leads/agents
# Calls can have one of five different outcomes
calls["Call Outcome"].value_counts()

CALL BACK LATER    1816
INTERESTED         1296
NOT INTERESTED     1211
ANSWER MACHINE      514
DEAD LINE           163
Name: Call Outcome, dtype: int64

In [6]:
# Rows in calls table uniquely determined by call number
calls["Call Number"].unique().size

5000

In [7]:
leads.head()

,Name,Phone Number,Region,Sector,Age
0,Isabela MEZA,175718505368,north-west,wholesale,19
1,Deangelo LEE,937521423043,north-west,retail,38
2,Rosia MENDEZ,403640999962,midlands,agriculture,40
3,Jeremiah GALLOWAY,946740713605,scotland,food,23
4,Sarah POPE,264176984341,midlands,retail,18


In [8]:
leads.tail()

,Name,Phone Number,Region,Sector,Age
9989,Anjanette MENDEZ,203652895277,midlands,wholesale,55
9990,Michele TRAN,572582587665,midlands,retail,38
9991,Jeannette ROBERSON,744364544246,south-west,retail,52
9992,Keaton BARRY,527132861814,north-west,retail,30
9993,Dallas MOSLEY,450781977953,south,food,97


In [9]:
leads.shape

(9994, 5)

In [10]:
# Rows in leads table uniquely determined by phone number
leads["Phone Number"].unique().size

9994

In [11]:
signups.shape

(768, 2)

In [12]:
# Rows in signups table uniquely determined by lead name
signups["Lead"].unique().size

768

In [13]:
signups["Approval Decision"].value_counts()

REJECTED    511
APPROVED    257
Name: Approval Decision, dtype: int64

In [14]:
signups.head()

,Lead,Approval Decision
0,Tyree TERRY,APPROVED
1,Ansel WOOD,REJECTED
2,Ludwig DIAZ,APPROVED
3,Mack ARELLANO,APPROVED
4,Judy HENDRICKS,REJECTED


In [15]:
signups.tail()

,Lead,Approval Decision
763,April EATON,REJECTED
764,Kaycee BAXTER,REJECTED
765,Channing SOTO,APPROVED
766,August PACHECO,REJECTED
767,Lindsey DAVILA,APPROVED


In [16]:
# check for null values
print(calls.isnull().values.any())
print(leads.isnull().values.any())
print(signups.isnull().values.any())

False
False
False


## Questions

1 Which agent made the most calls? 

In [17]:
calls.groupby("Agent")["Call Number"].agg("count").sort_values(ascending=False)

Agent
orange    2234
red       1478
black      750
green      339
blue       199
Name: Call Number, dtype: int64

Agent Orange made the most calls

2 For the leads that received one or more calls, how many calls were received on average?

In [18]:
calls_leads = calls.merge(leads, on="Phone Number", how='left')

calls_leads.groupby("Phone Number")["Call Number"].agg("count").mean()

1.839587932303164

Average number of call received by leads is 1.84

3 For the leads that signed up, how many calls were received, on average?

In [19]:
calls_leads.head()

,Phone Number,Call Outcome,Agent,Call Number,Name,Region,Sector,Age
0,83473306392,NOT INTERESTED,orange,0,Michell GUTIERREZ,wales,retail,50
1,762850680150,CALL BACK LATER,orange,1,Shanita FISHER,north-west,consultancy,23
2,476309275079,NOT INTERESTED,orange,2,Kaye DURAN,north-east,retail,34
3,899921761538,CALL BACK LATER,red,3,Barbie SALINAS,north-east,consultancy,23
4,906739234066,CALL BACK LATER,orange,4,Lisette NICHOLSON,south-west,food,25


In [20]:
all_data = pd.merge(calls_leads, signups, how="left", left_on="Name", right_on="Lead")

all_data[all_data["Approval Decision"] == "APPROVED"].groupby("Phone Number")["Call Number"].agg("count").mean()

1.9961089494163424

Average number of call received by leads that signed up is 1.996

4 Which agent had the most signups? Which assumptions did you make? (note that there is a many-to-one relationship between calls and leads)

In [21]:
# have two scenarios to consider:
# (1) Lead only has one call with to a single agent
# (2) Leads has multiple calls with either the same agent or multiple agents

# Assumption: the agent who made the last (or in the case of (1) only) call to an lead is assigend that signup. 
# This is because it is the only easy way of unambiguously assigning a signup to an agent on a per-call basis.
approved = all_data[all_data["Approval Decision"] == "APPROVED"]

# This is equivalent to assigning the signup to agent who's call has call outcome "interested"
last_call = approved.sort_values(["Phone Number", "Call Number"]).drop_duplicates(subset=["Phone Number"], keep="last")

print("Number of signups is equal to number of calls with call outcome \"interested\"")
print("Number of calls to distinct leads", last_call["Call Outcome"].value_counts()[0])
print("Number of calls with call outcome \"interested\"", approved[approved["Call Outcome"] == "INTERESTED"].shape[0])

last_call.groupby("Agent")["Call Number"].agg("count").sort_values(ascending=False)

Number of signups is equal to number of calls with call outcome "interested"
Number of calls to distinct leads 257
Number of calls with call outcome "interested" 257


Agent
red       117
orange     93
green      22
blue       15
black      10
Name: Call Number, dtype: int64

Agent red had the most signups of 117.

5 Which agent had the most signups per call?

In [22]:
# following from question 4, assign signup to agent who made last call to lead

signups_by_agent = pd.DataFrame({
        "total": all_data[["Agent", "Call Number"]].groupby("Agent")["Call Number"].agg("count"),
        "signups": last_call[["Agent", "Call Number"]].groupby("Agent")["Call Number"].agg("count")
    }).reset_index()

signups_by_agent["signups_per_call"] = signups_by_agent["signups"] / signups_by_agent["total"]

signups_by_agent.sort_values("signups_per_call", ascending=False)

,Agent,signups,total,signups_per_call
4,red,117,1478,0.079161
1,blue,15,199,0.075377
2,green,22,339,0.064897
3,orange,93,2234,0.041629
0,black,10,750,0.013333


The agent with most signups per call was agent red at 7.92%

6 Was the variation between the agents’ signups-per-call statistically significant? Why? 

In [23]:
agent_1 = signups_by_agent.iloc[0][["signups", "total"]].values.tolist()
agent_2 = signups_by_agent.iloc[1][["signups", "total"]].values.tolist()
agent_3 = signups_by_agent.iloc[2][["signups", "total"]].values.tolist()
agent_4 = signups_by_agent.iloc[3][["signups", "total"]].values.tolist()
agent_5 = signups_by_agent.iloc[4][["signups", "total"]].values.tolist()

data = [agent_1, agent_2, agent_3, agent_4, agent_5]
stats.chi2_contingency(data)

(48.582217852739156,
 7.136303403672277e-10,
 4,
 array([[  37.1542705 ,  722.8457295 ],
        [  10.46186038,  203.53813962],
        [  17.64827849,  343.35172151],
        [ 113.7605098 , 2213.2394902 ],
        [  77.97508084, 1517.02491916]]))

The pval above is much smaller than 0.05, indicating that the variation of signups-per-call between agents is not indenpentdently distributed: chi2(4) >= 48.6, p = 7.2e-10. In other words, certain agents have statistically signifcant better performance compared other agents.

7 A lead from which region is most likely to be “interested” in the product?

In [24]:
leads_by_region = pd.DataFrame({
                "total": all_data[["Name", "Region"]].drop_duplicates().groupby("Region")["Name"].agg("count"), 
                "interested": all_data[all_data["Call Outcome"] == "INTERESTED"][["Name", "Region"]].drop_duplicates().groupby("Region")["Name"].agg("count")
            }).reset_index()

leads_by_region["prop_interested"] = leads_by_region["interested"] / leads_by_region["total"]

leads_by_region.sort_values("prop_interested", ascending=False)

,Region,interested,total,prop_interested
0,london,56,74,0.756757
3,north-west,365,635,0.574803
8,south-west,161,288,0.559028
7,south-east,136,290,0.468966
2,north-east,139,300,0.463333
4,northern-ireland,40,87,0.459770
5,scotland,137,306,0.447712
6,south,62,145,0.427586
1,midlands,150,429,0.349650
9,wales,50,164,0.304878


Region with highest proportion of interested leads is London with 75.7% of leads interested.

8 A lead from which sector is most likely to be "interested" in the product? 

In [25]:
leads_by_sector = pd.DataFrame({
                "total": all_data[["Name", "Sector"]].drop_duplicates().groupby("Sector")["Name"].agg("count"), 
                "interested": all_data[all_data["Call Outcome"] == "INTERESTED"][["Name", "Sector"]].drop_duplicates().groupby("Sector")["Name"].agg("count")
            }).reset_index()

leads_by_sector["prop_interested"] = leads_by_sector["interested"] / leads_by_sector["total"]

leads_by_sector.sort_values("prop_interested", ascending=False)

,Sector,interested,total,prop_interested
2,consultancy,301,462,0.651515
6,wholesale,233,396,0.588384
3,entertainment,135,254,0.531496
4,food,261,582,0.448454
0,agriculture,30,71,0.422535
5,retail,290,792,0.366162
1,construction,46,161,0.285714


Sector with highest proportion of interested leads is consultancy with 65.2% of leads interested.

9 Given a lead has already expressed interest and signed up, 

a signups from which region are most likely to be approved?

b Is this statistically significant? Why?

In [26]:
interested = all_data[all_data["Call Outcome"] == "INTERESTED"]

approval_by_region = pd.DataFrame({
                "total": interested[["Name", "Region"]].drop_duplicates().groupby("Region")["Name"].agg("count"), 
                "approved": interested[interested["Approval Decision"] == "APPROVED"][["Name", "Region"]].drop_duplicates().groupby("Region")["Name"].agg("count")
            }).reset_index()

approval_by_region["prop_approved"] = approval_by_region["approved"] / approval_by_region["total"]

approval_by_region.sort_values("prop_approved", ascending=False)

,Region,approved,total,prop_approved
5,scotland,37,137,0.270073
3,north-west,95,365,0.260274
7,south-east,29,136,0.213235
6,south,12,62,0.193548
1,midlands,26,150,0.173333
8,south-west,25,161,0.155280
4,northern-ireland,6,40,0.150000
2,north-east,20,139,0.143885
9,wales,5,50,0.100000
0,london,2,56,0.035714


Signups from Scotland are most likely to be approved

In [27]:
# compared between Scotland and all other regions

count = [
    approval_by_region[approval_by_region["Region"] == "scotland"]["approved"].values[0],
    approval_by_region[approval_by_region["Region"] != "scotland"]["approved"].sum()
]

nobs = [
    approval_by_region[approval_by_region["Region"] == "scotland"]["total"].values[0],
    approval_by_region[approval_by_region["Region"] != "scotland"]["total"].sum()
]

zstat, pval = proportions_ztest(count, nobs)
print(zstat, pval)

2.2279110466972245 0.025886447392087147


In [28]:
stats.norm.cdf(zstat)

0.9870567763039564

In [29]:
stats.norm.ppf(1-(0.05/2))

1.959963984540054

The z-score of 2.22 is large than the critical value at 95% confidence of 1.96, indicating that there was a larger proportion of signups in Scotland compared to other regions, which is statistically significant given that pval < 0.05.

10 Suppose you wanted to pick the 1000 leads most likely to sign up (who have not been called so far), based only on age, sector and region.

a What criteria would you use to pick those leads

In [30]:
# based on the features listed above (age, sector, and region) want to find historic data of leads who have gone on
# to signup with an approval decision of "approved".
# the target label in this case is the "Approval Decision" from the signups table

# historic data of leads who have been called and feature on the signup table form the training dataset
historic_data = all_data[~all_data["Approval Decision"].isnull()][["Name", "Region", "Sector", "Age", "Approval Decision"]].drop_duplicates()

# age is a numeric feature
# sector and region are categorical so need to be converted into dummy variables
historic_data = pd.get_dummies(historic_data, columns=['Region', 'Sector'])

# need to convert "Approval Decision" into binary values
historic_data["Approval Decision"] = historic_data["Approval Decision"].apply(lambda d: 1 if d == "APPROVED" else 0)

# "Name" was just required to remove duplicates
historic_data = historic_data.drop("Name", axis=1)

X = historic_data.drop("Approval Decision", axis=1)
y = historic_data["Approval Decision"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=34)

# perform normalisation for ages feature
scaler = MinMaxScaler(feature_range = (0,1))
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

clf = LogisticRegression(random_state=34)

# select k best features using chi-squared test statistic
# choose k which maximised chosen evaluation metric of choice, which was precision - see below for discussion
# for sake of time I just ran this multiple times to find smallest k that maximised precision
selector = SelectKBest(chi2, k=8)

log = make_pipeline(selector, clf)
log.fit(X_train, y_train)
y_pred = log.predict(X_test)

In [31]:
# find names of selected features
selected_features = []

for tf, feature in zip(selector.get_support(), X.columns.values):
    if tf:
        selected_features.append(feature)

print(selected_features)

['Region_london', 'Region_north-west', 'Region_scotland', 'Region_wales', 'Sector_construction', 'Sector_consultancy', 'Sector_retail', 'Sector_wholesale']


In [32]:
# as the target classes are imbalanced makes sense not to use accuracy
y_train.value_counts()

0    357
1    180
Name: Approval Decision, dtype: int64

In [33]:
# I think for the purpose of the assignmnent, it makes sense to optimise for precision
# this is because we want to maximise the chance that an identified lead will actually
# sign up.
precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted', labels=np.unique(y_pred))

# i.e. approx. 3 out of every 4 leads correctly identified as going on to complete
print(precision)

0.7257309941520468


b In what sense are those an optimal criteria set?

The set of k best features, according to chi-squared test, which were found to maximise the precision, the chosen evaluation metric.

c How many signups would you expect to get based on those called leads, assuming they were being called by random agents?

In [34]:
# look at leads who have not been called
leads_to_target = all_data[all_data["Approval Decision"].isnull()][["Age", "Sector", "Region"]]
leads_to_target = pd.get_dummies(leads_to_target, columns=['Region', 'Sector'])
leads_to_target = leads_to_target[selected_features]

# get predicted probability for all classes
predictions = clf.predict_proba(leads_to_target)

predictions[predictions[:,0].argsort()][:1000]

array([[0.19909361, 0.80090639],
       [0.19909361, 0.80090639],
       [0.19909361, 0.80090639],
       ...,
       [0.38798826, 0.61201174],
       [0.38798826, 0.61201174],
       [0.38798826, 0.61201174]])

From above, find that the top 1000 leads most likely to signup - by predicted probability class - are more likely to signup than not. Or in other words, according the the classifier, there exists a subset of at least 1000 leads who are predicted to signup.

d If you could choose the agents to make those calls, who would you choose? Why?

Without making further assumptions about how a given agent performs in relation to the criteria found above, I would choose agents red and blue as they achieved a significantly higher signup rate than other agents. This is based on their performance in historic data.